# PROMETHEE Method
## **P**reference **R**anking **O**rganisation **M**ethod for **E**nrichment **E**valuations

PROMETHEE (Preference Ranking Organisation Method for Enrichment Evaluations) is a multi-criteria decision analysis (MCDA) method developed by Jean-Pierre Brans in 1982. This notebook focuses on **PROMETHEE II**, the most commonly used variant that provides a complete ranking of alternatives.

### What makes PROMETHEE II special:
- **Complete ranking**: Every alternative gets a definitive position
- **Pairwise comparisons**: Compares alternatives two at a time
- **Net flow approach**: Uses a single score (net flow) for final ranking
- **Preference functions**: Captures different types of preferences between alternatives
- **Transparent process**: Easy to understand and explain

### Basic Concept:
PROMETHEE II works by:
1. Comparing each pair of alternatives on each criterion
2. Using preference functions to quantify these comparisons
3. Aggregating preferences across all criteria using weights
4. Calculating a net flow score for each alternative
5. Ranking alternatives based on their net flow scores

## Mathematical Foundation

PROMETHEE is based on pairwise comparisons of alternatives. For each criterion, the method evaluates how much one alternative is preferred over another using **preference functions**.

### Core Components:

1. **Decision Matrix**: $A = [a_{ij}]_{m \times n}$ where:
   - $m$ = number of alternatives
   - $n$ = number of criteria
   - $a_{ij}$ = performance of alternative $i$ on criterion $j$

2. **Weights**: $w_j$ representing the relative importance of criterion $j$, where $\sum_{j=1}^{n} w_j = 1$

3. **Preference Function**: $P_j(a, b)$ indicating the preference degree of alternative $a$ over alternative $b$ for criterion $j$
   - Range: $[0, 1]$
   - $P_j(a, b) = 0$ means no preference
   - $P_j(a, b) = 1$ means strict preference

## Preference Functions - Starting Simple

PROMETHEE uses preference functions to quantify how much one alternative is preferred over another. We'll start with the two most basic types:

### Type 1: Usual Criterion (Step Function) - The Simplest
$$P(d) = \begin{cases} 0 & \text{if } d \leq 0 \\ 1 & \text{if } d > 0 \end{cases}$$

**When to use**: When any positive difference means strict preference
**Example**: Pass/Fail criteria, Yes/No decisions
**Parameters**: None required

### Type 5: Linear Preference - Most Intuitive
$$P(d) = \begin{cases} 0 & \text{if } d \leq 0 \\ \frac{d}{p} & \text{if } 0 < d \leq p \\ 1 & \text{if } d > p \end{cases}$$

**When to use**: When preference increases linearly with performance difference
**Example**: Cost differences, performance scores
**Parameters**: $p$ (maximum difference for strict preference)

---

### Other Preference Function Types (Advanced)

**Type 2: Quasi-criterion** - Ignores small differences
**Type 3: Linear with Indifference** - Combines indifference zone with linear growth  
**Type 4: Level Criterion** - Three-level preference (indifference, weak, strict)
**Type 6: Gaussian** - Smooth S-shaped preference curve

*We'll focus on Types 1 and 5 for our initial implementation, then explore others later.*

## PROMETHEE II Algorithm - Step by Step

### Step 1: Set Up the Problem
- Decision matrix with alternatives (rows) and criteria (columns)
- Criteria weights that sum to 1
- Choose preference function type for each criterion

### Step 2: Calculate Preference Degrees
For each pair of alternatives $(a, b)$ and each criterion $j$:

**Difference**: $d_j(a,b) = g_j(a) - g_j(b)$
**Preference**: $P_j(a,b) = F_j[d_j(a,b)]$

where $F_j$ is the preference function for criterion $j$.

### Step 3: Calculate Overall Preference
Combine all criteria using weights:
$$\pi(a,b) = \sum_{j=1}^{n} w_j \cdot P_j(a,b)$$

### Step 4: Calculate Net Flow (The Key Step)
For each alternative $a$:

**Positive Flow** (how much $a$ dominates others):
$$\phi^+(a) = \frac{1}{m-1} \sum_{x \neq a} \pi(a,x)$$

**Negative Flow** (how much others dominate $a$):
$$\phi^-(a) = \frac{1}{m-1} \sum_{x \neq a} \pi(x,a)$$

**Net Flow** (final score):
$$\phi(a) = \phi^+(a) - \phi^-(a)$$

### Step 5: Final Ranking
Rank alternatives by net flow: **Higher $\phi(a)$ = Better alternative**

## Why PROMETHEE II?

We focus on **PROMETHEE II** because it's the most practical variant:

### PROMETHEE II Characteristics
- **Complete ranking**: Every alternative gets a clear position (1st, 2nd, 3rd, etc.)
- **Single score**: Each alternative has one net flow score
- **No ties or incomparability**: Unlike PROMETHEE I, there are no ambiguous results
- **Easy interpretation**: Higher score = better alternative

### Other PROMETHEE Variants (Brief Overview)

**PROMETHEE I (Partial Ranking)**
- May leave some alternatives incomparable
- Uses both positive and negative flows separately
- More conservative but less decisive

**PROMETHEE III & IV**
- Handle uncertainty and continuous criteria
- More complex, less commonly used

## Advantages of PROMETHEE II

- **Transparent**: Each step is clear and explainable
- **Flexible**: Works with different types of criteria
- **Robust**: Adding or removing alternatives doesn't change existing comparisons
- **Visual**: Results can be displayed graphically
- **Practical**: Provides definitive rankings for decision making

## Limitations to Consider

- **Subjective elements**: Requires choosing preference functions and weights
- **Compensation**: Good performance on one criterion can offset poor performance on another
- **Parameter sensitivity**: Results may change with different preference function parameters

## Real-World Applications

### Business & Management
- Supplier selection
- Investment portfolio decisions
- Employee performance evaluation
- Strategic planning
- Project selection

### Engineering & Technology
- Equipment selection
- Technology assessment
- Quality control
- Design optimization
- Maintenance planning

### Environmental & Sustainability
- Environmental impact assessment
- Renewable energy project evaluation
- Waste management strategies
- Water resource allocation

### Public Sector
- Policy alternative evaluation
- Urban planning decisions
- Healthcare resource allocation
- Transportation planning

## Manufacturing Equipment Selection Example

We'll evaluate **4 CNC machines** for a manufacturing company using **5 criteria**:

### Problem Setup
**Alternatives**: A1, A2, A3, A4 (four different CNC machines)

**Criteria**:
1. **Cost** (minimize): Purchase price in thousands USD
2. **Precision** (minimize): Tolerance in micrometers (lower = better accuracy)  
3. **Speed** (maximize): Parts per hour
4. **Reliability** (maximize): Mean time between failures (hours)
5. **Power Consumption** (minimize): kW per hour

This is a realistic scenario where a manufacturing company needs to select the best CNC machine considering multiple conflicting objectives.

In [4]:
# Imports and Setup
import numpy as np
import pandas as pd

In [5]:
# Step 1: Create Decision Matrix
# CNC Machine alternatives with performance data

alternatives = [
    "A1_BasicCNC",
    "A2_PrecisionCNC",
    "A3_HighSpeedCNC",
    "A4_IndustrialCNC",
]

criteria_data = {
    "Cost": [120, 180, 150, 200],  # Thousands USD (minimize)
    "Precision": [10, 5, 8, 3],  # Micrometers (maximize - lower is better)
    "Speed": [25, 20, 40, 35],  # Parts per hour (maximize)
    "Reliability": [2000, 3500, 2800, 4200],  # MTBF in hours (maximize)
    "Power_Consumption": [15, 12, 18, 10],  # kW per hour (minimize)
}

# Create decision matrix
decision_matrix = pd.DataFrame(criteria_data, index=alternatives)

decision_matrix

,Cost,Precision,Speed,Reliability,Power_Consumption
A1_BasicCNC,120,10,25,2000,15
A2_PrecisionCNC,180,5,20,3500,12
A3_HighSpeedCNC,150,8,40,2800,18
A4_IndustrialCNC,200,3,35,4200,10


In [7]:
# Criteria information
criteria_info = {
    "Cost": {"type": "minimize", "unit": "k PHP"},
    "Precision": {
        "type": "minimize",
        "unit": "μm",
    },  # Note: Lower precision value = better
    "Speed": {"type": "maximize", "unit": "parts/hour"},
    "Reliability": {"type": "maximize", "unit": "MTBF hours"},
    "Power_Consumption": {"type": "minimize", "unit": "kW/hour"},
}

print("Criteria Information:")
for criterion, info in criteria_info.items():
    print(f"• {criterion}: {info['type']} ({info['unit']})")

print(f"\nAlternatives: {len(alternatives)} CNC machines")
print(f"Criteria: {len(criteria_info)} evaluation factors")

Criteria Information:
• Cost: minimize (k PHP)
• Precision: minimize (μm)
• Speed: maximize (parts/hour)
• Reliability: maximize (MTBF hours)
• Power_Consumption: minimize (kW/hour)

Alternatives: 4 CNC machines
Criteria: 5 evaluation factors


In [ ]:
# Step 2: Define Criteria Weights and Preference Functions

# Criteria weights (must sum to 1)
weights = {
    "Cost": 0.25,
    "Precision": 0.30,
    "Speed": 0.20,
    "Reliability": 0.20,
    "Power_Consumption": 0.05,
}

print("Criteria Weights:")
total_weight = 0
for criterion, weight in weights.items():
    print(f"• {criterion}: {weight:.2f} ({weight * 100:.0f}%)")
    total_weight += weight
print(f"Total weight: {total_weight:.2f} ✓")

# Preference function parameters
# For this example, we'll use Linear Preference (Type 5) for most criteria
preference_params = {
    "Cost": {"type": "linear", "p": 50},  # p = 50k USD threshold
    "Precision": {"type": "linear", "p": 5},  # p = 5 μm threshold
    "Speed": {"type": "linear", "p": 20},  # p = 20 parts/hour threshold
    "Reliability": {"type": "linear", "p": 1500},  # p = 1500 hours threshold
    "Power_Consumption": {"type": "linear", "p": 8},  # p = 8 kW threshold
}

print("\nPreference Function Parameters:")
for criterion, params in preference_params.items():
    print(
        f"• {criterion}: {params['type']} with p = {params['p']} {criteria_info[criterion]['unit']}"
    )

print("\nInterpretation:")
print(
    "• Linear preference: gradual increase in preference as performance difference grows"
)
print("• Parameter 'p': difference needed for maximum (strict) preference")

Criteria Weights:
• Cost: 0.25 (25%)
• Precision: 0.30 (30%)
• Speed: 0.20 (20%)
• Reliability: 0.20 (20%)
• Power_Consumption: 0.05 (5%)
Total weight: 1.00 ✓

Preference Function Parameters:
• Cost: linear with p = 50 k USD
• Precision: linear with p = 5 μm
• Speed: linear with p = 20 parts/hour
• Reliability: linear with p = 1500 MTBF hours
• Power_Consumption: linear with p = 8 kW/hour

Interpretation:
• Linear preference: gradual increase in preference as performance difference grows
• Parameter 'p': difference needed for maximum (strict) preference


In [8]:
# Step 3: Implement Preference Functions
def linear_preference(d: float, p: float) -> float:
    """
    Linear preference function (Type 5)
    Returns preference degree between 0 and 1
    """
    if d <= 0:
        return 0.0
    elif d >= p:
        return 1.0
    else:
        return d / p


def calculate_preference_degree(
    val_a: float, val_b: float, criterion: str, maximize: bool = True
) -> float:
    """
    Calculate preference degree of alternative a over alternative b for a given criterion
    """
    if maximize:
        d = val_a - val_b  # Positive if a is better than b
    else:
        d = val_b - val_a  # Positive if a is better than b (lower value)

    # Get preference function parameters
    params = preference_params[criterion]
    p = params["p"]

    # Apply preference function
    if params["type"] == "linear":
        return linear_preference(d, p)
    else:
        raise ValueError(
            f"Preference function type '{params['type']}' not implemented"
        )


# Test preference function with examples
print("Preference Function Examples:")
print("=" * 40)

# Example 1: Cost comparison (minimize criterion)
cost_a1, cost_a2 = 120, 180  # A1 vs A2
pref_degree = calculate_preference_degree(
    cost_a1, cost_a2, "Cost", maximize=False
)
print(f"Cost: A1 ({cost_a1}k) vs A2 ({cost_a2}k)")
print(f"Preference of A1 over A2: {pref_degree:.3f}")
print(f"→ A1 is {pref_degree * 100:.1f}% preferred over A2 on cost")

print()

# Example 2: Speed comparison (maximize criterion)
speed_a3, speed_a1 = 40, 25  # A3 vs A1
pref_degree = calculate_preference_degree(
    speed_a3, speed_a1, "Speed", maximize=True
)
print(f"Speed: A3 ({speed_a3}/hour) vs A1 ({speed_a1}/hour)")
print(f"Preference of A3 over A1: {pref_degree:.3f}")
print(f"→ A3 is {pref_degree * 100:.1f}% preferred over A1 on speed")

Preference Function Examples:
Cost: A1 (120k) vs A2 (180k)
Preference of A1 over A2: 1.000
→ A1 is 100.0% preferred over A2 on cost

Speed: A3 (40/hour) vs A1 (25/hour)
Preference of A3 over A1: 0.750
→ A3 is 75.0% preferred over A1 on speed


In [9]:
# Step 4: Calculate Pairwise Preference Matrices
n_alternatives = len(alternatives)
criteria_list = list(decision_matrix.columns)

# Store preference matrices for each criterion
preference_matrices = {}

print("Computing Pairwise Preferences...")
print("=" * 50)

for criterion in criteria_list:
    print(f"\nCriterion: {criterion}")

    # Determine if criterion should be maximized or minimized
    maximize = criteria_info[criterion]["type"] == "maximize"

    # Initialize preference matrix
    pref_matrix = np.zeros((n_alternatives, n_alternatives))

    # Calculate preferences for all pairs
    for i in range(n_alternatives):
        for j in range(n_alternatives):
            if i != j:  # Don't compare alternative with itself
                val_i = decision_matrix.iloc[i][criterion]
                val_j = decision_matrix.iloc[j][criterion]

                pref_degree = calculate_preference_degree(
                    val_i, val_j, criterion, maximize
                )
                pref_matrix[i][j] = pref_degree

    # Convert to DataFrame for better display
    pref_df = pd.DataFrame(
        pref_matrix, index=alternatives, columns=alternatives
    )

    preference_matrices[criterion] = pref_df

    print(f"Preference Matrix ({criterion}):")
    print(pref_df.round(3))
    print(f"→ {criterion} {'maximized' if maximize else 'minimized'}")

print(f"\n✓ Calculated preference matrices for {len(criteria_list)} criteria")
print(
    "Note: P(A,B) = preference degree of A over B (0 = no preference, 1 = strict preference)"
)

Computing Pairwise Preferences...

Criterion: Cost
Preference Matrix (Cost):
                  A1_BasicCNC  A2_PrecisionCNC  A3_HighSpeedCNC  \
A1_BasicCNC            0.0000           1.0000           0.6000   
A2_PrecisionCNC        0.0000           0.0000           0.0000   
A3_HighSpeedCNC        0.0000           0.6000           0.0000   
A4_IndustrialCNC       0.0000           0.0000           0.0000   

                  A4_IndustrialCNC  
A1_BasicCNC                 1.0000  
A2_PrecisionCNC             0.4000  
A3_HighSpeedCNC             1.0000  
A4_IndustrialCNC            0.0000  
→ Cost minimized

Criterion: Precision
Preference Matrix (Precision):
                  A1_BasicCNC  A2_PrecisionCNC  A3_HighSpeedCNC  \
A1_BasicCNC            0.0000           0.0000           0.0000   
A2_PrecisionCNC        1.0000           0.0000           0.6000   
A3_HighSpeedCNC        0.4000           0.0000           0.0000   
A4_IndustrialCNC       1.0000           0.4000           1.0000 

In [10]:
# Step 5: Calculate Overall Preference Matrix

# Aggregate preferences across all criteria using weights
overall_preference = np.zeros((n_alternatives, n_alternatives))

print("Aggregating Preferences Using Criteria Weights...")
print("=" * 50)

for criterion in criteria_list:
    weight = weights[criterion]
    pref_matrix = preference_matrices[criterion].values

    # Add weighted contribution of this criterion
    overall_preference += weight * pref_matrix

    print(f"• {criterion}: weight = {weight:.3f}")

# Convert to DataFrame
overall_pref_df = pd.DataFrame(
    overall_preference, index=alternatives, columns=alternatives
)

print("\nOverall Preference Matrix π(a,b):")
print(overall_pref_df.round(4))

print("\nInterpretation:")
print("• π(A1,A2) = overall preference degree of A1 over A2")
print("• Higher values indicate stronger preference")
print("• Matrix is asymmetric: π(A,B) ≠ π(B,A)")

# Show some key comparisons
print("\nKey Comparisons:")
for i in range(n_alternatives):
    for j in range(i + 1, n_alternatives):
        alt_i, alt_j = alternatives[i], alternatives[j]
        pref_ij = overall_pref_df.loc[alt_i, alt_j]
        pref_ji = overall_pref_df.loc[alt_j, alt_i]

        if pref_ij > pref_ji:
            stronger = alt_i
            diff = pref_ij - pref_ji
        else:
            stronger = alt_j
            diff = pref_ji - pref_ij

        print(f"• {stronger} preferred over the other by {diff:.3f}")

Aggregating Preferences Using Criteria Weights...
• Cost: weight = 0.250
• Precision: weight = 0.300
• Speed: weight = 0.200
• Reliability: weight = 0.200
• Power_Consumption: weight = 0.050

Overall Preference Matrix π(a,b):
                  A1_BasicCNC  A2_PrecisionCNC  A3_HighSpeedCNC  \
A1_BasicCNC            0.0000           0.3000           0.1688   
A2_PrecisionCNC        0.5188           0.0000           0.3108   
A3_HighSpeedCNC        0.3767           0.3500           0.0000   
A4_IndustrialCNC       0.6313           0.3758           0.5367   

                  A4_IndustrialCNC  
A1_BasicCNC                 0.2500  
A2_PrecisionCNC             0.1000  
A3_HighSpeedCNC             0.3000  
A4_IndustrialCNC            0.0000  

Interpretation:
• π(A1,A2) = overall preference degree of A1 over A2
• Higher values indicate stronger preference
• Matrix is asymmetric: π(A,B) ≠ π(B,A)

Key Comparisons:
• A2_PrecisionCNC preferred over the other by 0.219
• A3_HighSpeedCNC preferred 

In [11]:
# Step 6: Calculate PROMETHEE Flows and Final Ranking
print("PROMETHEE II Flow Calculations")
print("=" * 50)

# Initialize results dictionary
results = {}

for i, alt in enumerate(alternatives):
    # Positive flow: how much this alternative dominates others
    positive_flow = np.sum(
        [overall_pref_df.iloc[i, j] for j in range(n_alternatives) if i != j]
    ) / (n_alternatives - 1)

    # Negative flow: how much others dominate this alternative
    negative_flow = np.sum(
        [overall_pref_df.iloc[j, i] for j in range(n_alternatives) if i != j]
    ) / (n_alternatives - 1)

    # Net flow: final score (higher is better)
    net_flow = positive_flow - negative_flow

    results[alt] = {
        "Positive_Flow": positive_flow,
        "Negative_Flow": negative_flow,
        "Net_Flow": net_flow,
    }

    print(f"{alt}:")
    print(f"  • Positive Flow φ⁺: {positive_flow:.4f} (dominance over others)")
    print(f"  • Negative Flow φ⁻: {negative_flow:.4f} (dominated by others)")
    print(f"  • Net Flow φ:      {net_flow:.4f} (final score)")
    print()

# Create results DataFrame
results_df = pd.DataFrame(results).T

# Sort by Net Flow (descending - higher is better)
final_ranking = results_df.sort_values("Net_Flow", ascending=False)
final_ranking["Rank"] = range(1, len(final_ranking) + 1)

print("FINAL RANKING - PROMETHEE II Results")
print("=" * 50)
print(
    final_ranking[
        ["Net_Flow", "Positive_Flow", "Negative_Flow", "Rank"]
    ].round(4)
)

print("\nRECOMMENDATION:")
best_alternative = final_ranking.index[0]
best_score = final_ranking.iloc[0]["Net_Flow"]
print(f"Select {best_alternative} (Net Flow: {best_score:.4f})")

# Show ranking interpretation
print("\nRanking Interpretation:")
for i, (alt, row) in enumerate(final_ranking.iterrows()):
    rank = i + 1
    score = row["Net_Flow"]
    if rank == 1:
        print(f"{rank}. {alt}: Best choice (φ = {score:.4f})")
    elif rank == len(final_ranking):
        print(f"{rank}. {alt}: Least preferred (φ = {score:.4f})")
    else:
        print(f"{rank}. {alt}: Rank {rank} (φ = {score:.4f})")

PROMETHEE II Flow Calculations
A1_BasicCNC:
  • Positive Flow φ⁺: 0.2396 (dominance over others)
  • Negative Flow φ⁻: 0.5089 (dominated by others)
  • Net Flow φ:      -0.2693 (final score)

A2_PrecisionCNC:
  • Positive Flow φ⁺: 0.3099 (dominance over others)
  • Negative Flow φ⁻: 0.3419 (dominated by others)
  • Net Flow φ:      -0.0321 (final score)

A3_HighSpeedCNC:
  • Positive Flow φ⁺: 0.3422 (dominance over others)
  • Negative Flow φ⁻: 0.3388 (dominated by others)
  • Net Flow φ:      0.0035 (final score)

A4_IndustrialCNC:
  • Positive Flow φ⁺: 0.5146 (dominance over others)
  • Negative Flow φ⁻: 0.2167 (dominated by others)
  • Net Flow φ:      0.2979 (final score)

FINAL RANKING - PROMETHEE II Results
                  Net_Flow  Positive_Flow  Negative_Flow  Rank
A4_IndustrialCNC    0.2979         0.5146         0.2167     1
A3_HighSpeedCNC     0.0035         0.3422         0.3388     2
A2_PrecisionCNC    -0.0321         0.3099         0.3419     3
A1_BasicCNC        -0.269